# Extracción de características *Bag of Words*

Primero importamos todas las librerías necesarias

In [ ]:
import pandas as pd
import numpy as np
import re
import string
import spacy
import gensim

pd.options.display.max_colwidth = None


Creamos un pequeño cuerpo de textos de ejemplo *(CORPUS)*

In [ ]:
corpus = ['El cielo es azul y bonito',
          'Me encanta el cielo azul, pero no el cielo plomizo',
          'Bonito cielo hacía ese día',
          'Hoy he desayunado huevos con jamón y tostadas',
          'Juan odia las tostadas y los huevos con jamón',
          'las tostadas de jamón están muy buenas']

## Limpieza del texto
Definimos una función simple de limpieza y normalización del texto y la aplicamos a nuestro corpus.

In [ ]:
nlp = spacy.load("es_core_news_sm")
def normalizar_doc(doc):
    '''Función que normaliza un texto cogiendo sólo
    las palabras en minúsculas mayores de 3 caracteres'''
    # separamos en tokens
    tokens = nlp(doc)
    # filtramos stopwords
    filtered_tokens = [t.lower_ for t in tokens if
                       len(t.text)>3 and
                       not t.is_punct]
    # juntamos de nuevo en una cadena
    doc = ' '.join(filtered_tokens)
    return doc

In [ ]:
#probamos la función
normalizar_doc(corpus[0])

In [ ]:
corpus[0]

In [ ]:
#aplicamos a todo el corpus
norm_corpus = [normalizar_doc(doc) for doc in corpus]
norm_corpus

In [ ]:
map(normalizar_doc, corpus)

In [ ]:
#alternativamente
list(map(normalizar_doc, corpus))

In [ ]:
for t in map(normalizar_doc, corpus):
    print(t)

# Librería `scikit-learn`
Implementamos el modelo Bag-of-Word (BoW) con `scikit-learn`

Contamos la frecuencia de aparición de los términos en cada documento, usando un vocabulario común. 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
cv.fit(norm_corpus) #también funcionaría cv.fit(map(normalizar_doc, corpus))

In [ ]:
type(cv)

In [ ]:
# obtenemos palabras únicas en el corpus
vocab = cv.get_feature_names_out()
vocab

In [ ]:
len(vocab)

In [ ]:
cv_matrix = cv.transform(norm_corpus)
cv_matrix.shape

In [ ]:
#matriz sparse
cv_matrix

In [ ]:
#sólo guarda info de las celdas no vacías
print(cv_matrix)

In [ ]:
#convertimos en matriz densa
cv_matrix = cv_matrix.toarray()
cv_matrix

Cada término único es una característica de la matriz generada:

In [ ]:
# mostramos vectores de características BoW del corpus
pd.DataFrame(cv_matrix, columns=vocab)

El modelo genera un diccionario con todas las palabras del vocabulario y asigna un índice único a cada palabra:

In [ ]:
cv.vocabulary_

In [ ]:
#id de las palabras del vocabulario
cv.vocabulary_.get('cielo')

In [ ]:
#si una palabra no está en el vocabulario...
cv.vocabulary_.get('lluvia')

### Aplicando el modelo a nuevos documentos
Cuando calculamos el vector BoW de un texto nuevo con el modelo no hay que volver a ajustar el vocabulario, por lo que los términos nuevos no se tendrán en cuenta:

In [ ]:
nuevo_corpus = ['El Cielo amenaza lluvia', 'Pedro desayuna tostadas de jamón con tomate']
cv_matrix_nueva = cv.transform(map(normalizar_doc, nuevo_corpus))
cv_matrix_nueva

In [ ]:
pd.DataFrame(cv_matrix_nueva.toarray(), columns=vocab)

### Modelos N-grams
Considera como términos del vocabulario cada secuencia de N palabras consecutivas que aparece en el texto (*n-gramas*).  
Por ejemplo para los *bigrams* del corpus (N=2):

In [ ]:
bv = CountVectorizer(ngram_range=(2,2))
bv_matrix = bv.fit_transform(norm_corpus)

In [ ]:
vocab_bigram = bv.get_feature_names_out()
vocab_bigram

In [ ]:
len(vocab_bigram)

In [ ]:
bv_matrix

In [ ]:
bv_matrix = bv_matrix.toarray()
pd.DataFrame(bv_matrix, columns=vocab_bigram)

Se puede establecer el rango de n-grams a `(1,2)` para obtener el conjunto de unigramas y bigramas del corpus.  
Para limitar el número de términos en el vocabulario del modelo BoW se puede limitar a los términos que aparecen en un mínimo de documentos con el parámetro `min_df`

In [ ]:
bv = CountVectorizer(ngram_range=(1,2), min_df=2)
bv_matrix = bv.fit_transform(norm_corpus)

bv_matrix = bv_matrix.toarray()
vocab_bigram = bv.get_feature_names_out()
pd.DataFrame(bv_matrix, columns=vocab_bigram)

In [ ]:
bv_matrix.shape

# Librería `Gensim`
Para trabajar con la librería `Gensim` es necesario transformar los documentos en una lista de tokens.

In [ ]:
def word_tokenize(text):
    return [token.text for token in nlp.make_doc(text)]

Convertimos nuestros texto de ejemplo en una lista de tokens y visualizamos todo el corpus:

In [ ]:
[word_tokenize(doc) for doc in norm_corpus]

Podemos definir una función para normalizar y al mismo tiempo tokenizar el texto:

In [ ]:
def normalizar_doc_tokenize(doc):
    '''Función que normaliza un texto cogiendo sólo
    las palabras en minúsculas mayores de 3 caracteres'''
    # separamos en tokens
    tokens = nlp(doc)
    # filtramos stopwords
    filtered_tokens = [t.lower_ for t in tokens if
                       len(t.text)>3 and
                       not t.is_punct]

    return filtered_tokens

In [ ]:
normalizar_doc_tokenize(corpus[0])

In [ ]:
tokenized_corpus = [normalizar_doc_tokenize(doc) for doc in corpus]
tokenized_corpus

Forma alternativa de crear el corpus tokenizado:

In [ ]:
list(map(normalizar_doc_tokenize, corpus))

## Modelo Bag of Words
Se pasará al modelo de Gensim como:

In [ ]:
from gensim.corpora import Dictionary

diccionario = Dictionary(tokenized_corpus) #alternativamente Dictionary(map(normalizar_doc_tokenize, corpus))

In [ ]:
diccionario

El ID de cada palabra del diccionario se obtiene con:

In [ ]:
diccionario.token2id #diccionario de id de cada palabra

In [ ]:
len(diccionario.token2id)

In [ ]:
diccionario.id2token #diccionario de palabras para cada ID

La librería `gensim` crea la matriz BoW con otro formato. A cada palabra distinta del corpus se le asigna un ID único. Por cada documento se genera una lista de tuplas (ID, frecuencia) con la frecuencia de aparición de cada palabra:

In [ ]:
diccionario.doc2bow(tokenized_corpus[0])

In [ ]:
diccionario.token2id['plomizo'] #ID de cada término

In [ ]:
diccionario[5] #término correspondiente a una ID

Creación de la matriz BoW

In [ ]:
mapped_corpus = [diccionario.doc2bow(text)
                 for text in tokenized_corpus]

In [ ]:
mapped_corpus

Por ejemplo el primer documento:

In [ ]:
for (i, tf) in mapped_corpus[0]:
    print(f"{diccionario[i]}: {tf}")

In [ ]:
#frec. de documentos de cada token
type(diccionario.dfs)

In [ ]:
diccionario.dfs

In [ ]:
for i in diccionario.dfs:
    print(f"{diccionario[i]}: {diccionario.dfs[i]}")

In [ ]:
#frec. de aparición de cada token
for i in diccionario.cfs:
    print(f"{diccionario[i]}: {diccionario.cfs[i]}")

In [ ]:
#alternativamente su usamos un generador
mapped_corpus = [diccionario.doc2bow(d) for d in map(normalizar_doc_tokenize, corpus)]

## Aplicación de los modelos a nuevos textos
Para aplicar un modelo BoW o TF-IDF a un nuevo documento hay que utilizar los modelos ya entrenados en `gensim` sobre el corpus original
### Modelo BoW

In [ ]:
tokenized_nuevo_corpus = [normalizar_doc_tokenize(doc) for doc in nuevo_corpus]

mapped_nuevo_corpus = [diccionario.doc2bow(text)
                 for text in tokenized_nuevo_corpus]

mapped_nuevo_corpus

In [ ]:
tokenized_nuevo_corpus

In [ ]:
for (i, tf) in mapped_nuevo_corpus[1]:
    print(f"{diccionario[i]}: {tf}")

In [ ]:
#Más pythonico con 'map'
diccionario = Dictionary(map(normalizar_doc_tokenize, corpus))
list(map(diccionario.doc2bow, map(normalizar_doc_tokenize, nuevo_corpus)))

In [ ]:
#o mejor incluso
list(map(lambda x: diccionario.doc2bow(normalizar_doc_tokenize(x)), nuevo_corpus))